## Implement GreedyMotifSearch
http://rosalind.info/problems/ba2d

In [1]:
with open("rosalind_ba2d_1.txt", "r") as f:
        k, t = map(int, f.readline().split())
        DNA = [line.strip() for line in f]
print(k)
print(t)
print(DNA)

3
5
['GGCGTTCAGGCA', 'AAGAATCAGTCA', 'CAAGGAGTTCGC', 'CACGTCAATCAC', 'CAATAATATTCG']


In [133]:
### Getting hamming distance between a k_mer and the DNA string
### import hamming distance from BA2C.ipynb
def Hamming_Distance(k_mer_pat,DNA_string):
    dist = 0  ## To track distance between k-mer and string
    minimum_distace = [] ### To find minimum distance from all the distances
    for i in range(len(DNA_string) - len(k_mer_pat) + 1): ## Lopping through string using a window of k_mer size
        for j in range(len(k_mer_pat)):               ## Looping for k_mer length times
            if k_mer_pat[j] != DNA_string[i:i+len(k_mer_pat)][j]: ## Compare each k_mer character with a character in a string
                dist = dist + 1 ## If a character is mismatched then increment distance
#         print(dist)
        minimum_distace.append(dist) ## Append distance value to a list
        dist = 0
    return min(minimum_distace) ## Return a lowest value from the list which is the lowest hamming distance


In [134]:
### import BA2C.ipynb
from operator import itemgetter
def ProfileMostProableMotif(DNA,k_mer,profile):
    position = {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}
    k_mer_1 = k_mer
    total_probability = []
    for i in range(len(DNA)-k_mer_1+1): ### Looping through DNA string
        k_mer_seq = DNA[i:k_mer] ## Getting k_mer sequences
        k_mer = k_mer + 1 ## To get new k_mer sequence
        initial_probability = 1
        for x in range(len(k_mer_seq)): ## Looping through each k_mer
            letter = k_mer_seq[x] ## Getting letter from k_mer
            proability = profile[position[letter]][x] ## Getting the proabality of that letter at that position from the profile matrix
            initial_probability = float(initial_probability) * float(proability) ## Multiplying with the initial proability
        total_probability.append((k_mer_seq,initial_probability)) ## APpendind all the multiplied proabilities to a list
#     print(max(total_probability, key=itemgetter(1))[0]) ## Getting the k-mer with the highest proabality value
    return max(total_probability, key=itemgetter(1))[0]

In [135]:
# ## Helper function to generate profile matrix
def Profile(gerated_motifs, k):
    profile_matrix = []
    for i in range(4):
        profile_matrix.append([0.0] * k) ## Creating a matrix or list with all 0's in each position for all 4 bases
    total_motifs = len(gerated_motifs) ## Getting the length of motifs
    for j in range(k):
        motif_counting = {"A" : 0, "C" : 0, "G" : 0, "T" : 0} # Creating a disctionary for all 4 bases with 0 as a key value
        for i in gerated_motifs:
            motif_counting["A"] += i[j].count("A") ## Counting a base in a motif and adding value to a dictionary if the base is A
            motif_counting["C"] += i[j].count("C") ## Counting a base in a motif and adding value to a dictionary if the base is C
            motif_counting["G"] += i[j].count("G") ## Counting a base in a motif and adding value to a dictionary if the base is G
            motif_counting["T"] += i[j].count("T") ## Counting a base in a motif and adding value to a dictionary if the base is T
        profile_matrix[0][j] = motif_counting["A"] / total_motifs ## Dividing each value in the dictinary with total motif length
        profile_matrix[1][j] = motif_counting["C"] / total_motifs
        profile_matrix[2][j] = motif_counting["G"] / total_motifs
        profile_matrix[3][j] = motif_counting["T"] / total_motifs
#         print("Profile",profile_matrix)
    return profile_matrix ## Returning final matrix


In [136]:
test = []
def ScoreFunction(given_motifs):
    length = len(given_motifs[0]) ## Getting length of motifs in each list of motifs
    consensus_seq = []
    for j in range(length): ## Looping through length value
        position = {"A" : 0, "C" : 0, "G" : 0, "T" : 0} ## A dictionary with 0's as values
        for i in given_motifs: ## Looping through each motif
            position["A"] = position["A"] + i[j].count("A") ## Counting a base in a motif and adding value to a dictionary if the base is A
            position["C"] = position["C"] + i[j].count("C")
            position["G"] = position["G"] + i[j].count("G")
            position["T"] = position["T"] + i[j].count("T")
        maximum_position = max(position.values()) ## Getting maximum dictionary value 
        for key,value in position.items():## Looping through dictionary
            if value == maximum_position: ## If max value == maximum dictionary value 
                consensus_seq.append(key) ## Get corresponding key for that maximum value

                break
    

    consensus_seq = "".join(consensus_seq) ## Join all the keys
    
    score = 0
    for i in given_motifs:
        score = score + Hamming_Distance(i,consensus_seq) ## Calculate the hamming distance between each generated motif and the generated consesus sequence
    return score
                

In [139]:
def GreedyMotifSearchAlgorithm(k, t, DNA):
    # create list of best motifs from out of the first DNA string given
    best_motifs = []
    for seq in DNA:
        first = seq[0:k]
        best_motifs.append(first)
#     best_motifs = [seq[:k] for seq in dna]
    
    # iterate over kmers in first Dna string, create a motif list for each kmer
    first_seq = DNA[0]
    for start in range(len(first_seq) - k + 1):
        kmer = first_seq[start : start + k]
        # start motif list based on this kmer from first_seq
        motif = [kmer]
#         print(motif)
        
        # iterate over subsequent Dna strings, make  profile from them based on first_seq kmer
        for i in range(1, t):
            matrix = Profile(motif,k)
#             print("Profile",matrix)
            most_probable = ProfileMostProableMotif(DNA[i], k, matrix)
#             print("Most probable",most_probable)
            motif.append(most_probable)
            
        # score motif, replace if best
        if ScoreFunction(motif) < ScoreFunction(best_motifs):
            best_motifs = motif
    index = 0
    for x in best_motifs:
#         print(x)
        index = index + 1
#     print(index)
    return 0

In [140]:
GreedyMotifSearchAlgorithm(k,t,DNA)

0